In [2]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import mplcursors
import numpy as np
import pandas as pd
import sys
if '..' not in sys.path: sys.path.append('..')
from f_ring_util.f_ring import (compute_corrected_ew, 
                                fit_hg_phase_function, 
                                hg_func,
                                print_hg_params,
                                scale_hg_phase_function)

%matplotlib notebook

# Utility Functions

In [3]:
### READ EW STATS AND OBS_LIST RESTRICTIONS

def read_obs_list():
    global OBS_LIST
    OBS_LIST = pd.read_csv('../obs_list.csv', parse_dates=['Date'], index_col='Observation')
    
def read_cassini_ew_stats(filename):
    obsdata = pd.read_csv(filename, parse_dates=['Date'])
    obsdata = obsdata.join(OBS_LIST, on='Observation', rsuffix='_obslist')
    obsdata = obsdata[obsdata['For Photometry'] == 1]
    print(f'** SUMMARY STATISTICS - {filename} **')
    print('Unique observation names:', len(obsdata.groupby('Observation')))
    print('Total slices:', len(obsdata))
    print('Starting date:', obsdata['Date'].min())
    print('Ending date:', obsdata['Date'].max())
    print('Time span:', obsdata['Date'].max()-obsdata['Date'].min())
    obsdata['Mu'] = np.abs(np.cos(np.radians(obsdata['Mean Emission'])))
    obsdata['Mu0'] = np.abs(np.cos(np.radians(obsdata['Incidence'])))
    return obsdata


### CALCULATE QUANTILES


def limit_by_quant(obsdata, cutoff1, cutoff2):
    def xform_func(column):
        if quant2 is None:
            return [(None if z > quant1[column.name] else z) for z in column]
        return [(None if z > quant1[column.name] or
                         z < quant2[column.name] else z) for z in column]
    obsdata = obsdata.copy()
    group = obsdata.groupby('Observation')
    quant1 = group.quantile(cutoff1/100, numeric_only=True)['Normal EW Mean']
    quant2 = None
    if cutoff2 is not None:
        quant2 = group.quantile(cutoff2/100, numeric_only=True)['Normal EW Mean']
    xform = group['Normal EW Mean'].transform(xform_func)
    obsdata['_control'] = xform
    obsdata.dropna(inplace=True)
    return obsdata
    

### SINGLE PLOTS - POINTS

def plot_various_quants(obsdata, include_phase=True):
    """Choose various quantiles of NormalEW and plot w/phase curves."""
    fig, axs = plt.subplots(6, 2, figsize=(10, 12))
    for plot_num, (perc1, perc2, color) in enumerate(((100, None, 'black'),
                                                      ( 90, None, 'black'),
                                                      ( 80, None, 'black'),
                                                      ( 70, None, 'black'),
                                                      ( 60, None, 'black'),
                                                      ( 50, None, 'black'),
                                                      ( 40, None, 'black'),
                                                      ( 30, None, 'black'),
                                                      ( 20, None, 'black'),
                                                      ( 10, None, 'black'),
                                                      ( 80,   20, 'blue'),
                                                      ( 70,   30, 'blue'))):
        ax = axs[plot_num // 2, plot_num % 2]
        quant_obsdata = limit_by_quant(obsdata, perc1, perc2)
        ax.scatter(quant_obsdata['Mean Phase'], quant_obsdata['Normal EW Mean'], marker='o', 
                   s=5, color=color, alpha=1)
        title = f'{perc1} / {perc2}'
        if include_phase:
            params, _, _ = fit_hg_phase_function(2, None, quant_obsdata)
            xrange = np.arange(quant_obsdata['Mean Phase'].min(), quant_obsdata['Mean Phase'].max()+1)
            full_phase_model = hg_func(params, xrange)
            lcolor = 'black' if color != 'black' else 'green'
            total_scale = params[1] + params[3]
            w1 = params[1] / total_scale
            w2 = params[3] / total_scale
            if params[1] < params[3]:
                title  += f' (g1 = {params[2]:6.3f} @ {w2:5.3f} / g2 = {params[0]:6.3f})'
            else:
                title  += f' (g1 = {params[0]:6.3f} @ {w1:5.3f} / g2 = {params[2]:6.3f})'
            ax.plot(xrange, full_phase_model, '-', color=lcolor, lw=3)
            quant_obsdata_mean = quant_obsdata.groupby('Observation').mean(numeric_only=True)
            params_mean, _, _ = fit_hg_phase_function(2, None, quant_obsdata_mean)
            print(f'*** {perc1} / {perc2}: {color}')
            print_hg_params(params)
            print_hg_params(params_mean)
        ax.set_yscale('log')
        ax.set_xlim(0, 180)
        ax.set_xlabel('Phase Angle (°)')
        ax.set_ylabel('Normal EW')
        ax.set_title(title)
    plt.tight_layout()
    
    
### SINGLE PLOTS ON AN AXIS - PHASE CURVE

def _standard_alpha(obsdata):
    """Return alpha based on number of points to plot."""
    if len(obsdata) < 1000:
        return 1
    elif len(obsdata) < 10000:
        return 0.6
    else:
        return 0.3

def _add_hover(obsdata, p):
    """Add hover text to scatter points."""
    cursor = mplcursors.cursor(p, hover=True)
    @cursor.connect('add')
    def on_add(sel):
        row = obsdata.iloc[sel.target.index]
        sel.annotation.set(text=f"{row['Observation']} @ {row['Min Long']:.2f}\n"
                                f"(a={row['Mean Phase']:.0f}, e={row['Mean Emission']:.0f}, "
                                f"i={row['Incidence']:.2f})")
        
def plot_points_phase_curve(obsdata, params, used_obsdata=None, title=None, 
                            col='Normal EW Mean', ax=None, **kwargs):
    """Plot scattered used/unused EW points and fit phase curve."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 5))

    obsdata['Phase Model'] = hg_func(params, obsdata['Mean Phase'])
    
    if used_obsdata is not None:
        used_obsdata['Phase Model'] = hg_func(params, used_obsdata['Mean Phase'])
        obsdata['_alpha'] = 0.1
        obsdata['_color'] = 'red'
        obsdata.loc[used_obsdata.index, '_alpha'] = 0.3
        obsdata.loc[used_obsdata.index, '_color'] = 'black'
    else:
        obsdata['_alpha'] = 0.3
        obsdata['_color'] = 'black'
        
    p = ax.scatter(obsdata['Mean Phase'], obsdata[col], marker='o', 
                   s=5, color=obsdata['_color'], alpha=obsdata['_alpha'])
    _add_hover(obsdata, p)

    # Plot the phase model sampled at 1-degree intervals
    xrange = np.arange(obsdata['Mean Phase'].min(), obsdata['Mean Phase'].max()+1)
    full_phase_model = hg_func(params, xrange)
    ax.plot(xrange, full_phase_model, '-', color='green', lw=2)
    ax.set_yscale('log')
    ax.set_xlim(0, 180)
    ax.set_xlabel('Phase Angle (°)')
    ax.set_ylabel(col)
    if title is not None:
        plt.title(title)

def plot_heatmap_phase_curve(obsdata, params, title=None, col='Normal EW Mean', ax=None, **kwargs):
    """Plot binned heatmap and fit phase curve."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 5))
    
    obsdata['Phase Model'] = hg_func(params, obsdata['Mean Phase'])

    ymin = obsdata[col].min()
    ymax = obsdata[col].max()
    heatmap, xedges, yedges = np.histogram2d(obsdata['Mean Phase'],
                                             np.log10(obsdata[col]),
                                             bins=(90, 50),
                                             range=[[0,180], [np.log10(ymin), np.log10(ymax)]])
    heatmap = heatmap ** .25
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    ax.imshow(heatmap.T[::-1,:], extent=extent, cmap=cm.Greys, interpolation='nearest', aspect='auto')
    
    # Plot the phase model sampled at 1-degree intervals
    xrange = np.arange(obsdata['Mean Phase'].min(), obsdata['Mean Phase'].max()+1)
    full_phase_model = hg_func(params, xrange)
    ax.plot(xrange, np.log10(full_phase_model), '-', color='green', lw=2)
    ax.set_xlabel('Phase Angle (°)')
    ax.set_ylabel(col)
    if title is not None:
        plt.title(title)

def plot_points_phase_time(obsdata, params, title=None, time_fit=3, col='Normal EW Mean', ax=None, **kwargs):
    """Plot scattered EW points by time with fit time curve colored by phase."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 5))

    obsdata['Phase Model'] = hg_func(params, obsdata['Mean Phase'])

    time0 = np.datetime64('1970-01-01T00:00:00') # epoch
    obsdata['Date_secs'] = (obsdata['Date']-time0).dt.total_seconds()/86400
    obsdata['Phase Curve Ratio'] = obsdata[col] / obsdata['Phase Model']
    alpha = _standard_alpha(obsdata)
    p = ax.scatter(obsdata['Date'], obsdata['Phase Curve Ratio'], marker='o', s=5,
                   c=obsdata['Mean Phase'], cmap=cm.jet, alpha=alpha)
    _add_hover(obsdata, p)

    timecoeff = np.polyfit(obsdata['Date_secs'], obsdata['Phase Curve Ratio'], time_fit)
    timerange = np.arange(obsdata['Date_secs'].min(), obsdata['Date_secs'].max(), 100)
    timefit = np.polyval(timecoeff, timerange)
    ax.plot(timerange, timefit, '-', lw=2, color='green')
    ax.set_yscale('log')
    ax.set_xlabel('Date of Observation')
    ax.set_ylabel(f'{col} / Full Phase Model')
    if title is not None:
        plt.title(title)

    
### SINGLE PLOTS ON AN AXIS - RATIOS

def plot_ratio_vs(obsdata, params, vs, color_by, order, ax=None):
    """Plot scattered used/unused EW points vs. another parameter."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 5))

    obsdata['Phase Model'] = hg_func(params, obsdata['Mean Phase'])
    obsdata['Phase Curve Ratio'] = obsdata['Normal EW Mean'] / obsdata['Phase Model']
    
    alpha = _standard_alpha(obsdata)
    p = ax.scatter(obsdata[vs], obsdata['Phase Curve Ratio'], marker='o', 
                   s=5, c=obsdata[color_by], cmap=cm.jet, alpha=alpha)
    _add_hover(obsdata, p)

    coeff = np.polyfit(obsdata[vs], obsdata['Phase Curve Ratio'], order)
    xrange = np.linspace(obsdata[vs].min(), obsdata[vs].max(), 100, endpoint=True)
    fit = np.polyval(coeff, xrange)
    ax.plot(xrange, fit, '-', lw=2, color='green')

    ax.set_yscale('log')
    ax.set_xlabel(vs)
    ax.set_ylabel('Normal EW / Full Phase Model')

    
### COMBINED PLOTS
    
def plot_points_phase_curve_time(obsdata, params, title, used_obsdata=None, time_fit=3, **kwargs):
    """Plot 1) scatter+phase curve 2) time fit."""
    fig, axs = plt.subplots(2, 1, figsize=(8, 10))
    plot_points_phase_curve(obsdata, params, used_obsdata=used_obsdata, ax=axs[0], **kwargs)
    plot_points_phase_time(obsdata, params, time_fit=time_fit, ax=axs[1], **kwargs)
    plt.suptitle(title)
    plt.tight_layout()
    
def plot_heatmap_phase_curve_time(obsdata, params, title, time_fit=3, **kwargs):
    """Plot 1) heatmap+phase curve 2) time fit."""
    fig, axs = plt.subplots(2, 1, figsize=(8, 10))
    plot_heatmap_phase_curve(obsdata, params, ax=axs[0], **kwargs)
    plot_points_phase_time(obsdata, params, time_fit=time_fit, ax=axs[1], **kwargs)
    plt.suptitle(title)
    plt.tight_layout()
    
def plot_ratio_vs_mu_mu0(obsdata, params, title, color_by='Mean Phase', order=3):
    """Plot 1) NEW/Model vs Mu 2) NEW/Model vs Mu0."""
    fig, axs = plt.subplots(2, 1, figsize=(8, 10))
    plot_ratio_vs(obsdata, params, vs='Mu', color_by=color_by, order=order, ax=axs[0])
    plot_ratio_vs(obsdata, params, vs='Mu0', color_by=color_by, order=order, ax=axs[1])
    plt.suptitle(title)
    plt.tight_layout()


In [4]:
read_obs_list()
obsdata_60_0 = read_cassini_ew_stats('../data_files/cass_ew_60_0.csv')
print()
obsdata_0_1 = read_cassini_ew_stats('../data_files/cass_ew_0_1.csv')
obsdata_0_1_mean = obsdata_0_1.groupby('Observation').mean(numeric_only=True)

** SUMMARY STATISTICS - ../data_files/cass_ew_60_0.csv **
Unique observation names: 155
Total slices: 155
Starting date: 2004-06-20 19:15:28
Ending date: 2017-09-06 11:47:07
Time span: 4825 days 16:31:39

** SUMMARY STATISTICS - ../data_files/cass_ew_0_1.csv **
Unique observation names: 210
Total slices: 38467
Starting date: 2004-06-20 19:15:28
Ending date: 2017-09-07 21:51:55
Time span: 4827 days 02:36:27


# Phase Curves with Tau

In [10]:
cutoff = 50
obsdata_limited = limit_by_quant(obsdata_60_0, cutoff, None)
for tau in np.arange(0, 0.06, 0.001):
    params_limited, _, std = fit_hg_phase_function(2, None, obsdata_limited, col_tau=('Normal EW Mean', tau))
    print(f'{tau:.3f} -> {std:10.6f}')

0.000 ->   0.054319
0.001 ->   0.053612
0.002 ->   0.053561
0.003 ->   0.054077
0.004 ->   0.055061
0.005 ->   0.056419
0.006 ->   0.058062
0.007 ->   0.059917
0.008 ->   0.061920
0.009 ->   0.064025
0.010 ->   0.066192
0.011 ->   0.068394
0.012 ->   0.070609
0.013 ->   0.072824
0.014 ->   0.075026
0.015 ->   0.077208
0.016 ->   0.079366
0.017 ->   0.081495
0.018 ->   0.083594
0.019 ->   0.085663
0.020 ->   0.087700
0.021 ->   0.089706
0.022 ->   0.091682
0.023 ->   0.093627
0.024 ->   0.095542
0.025 ->   0.097428
0.026 ->   0.099287
0.027 ->   0.101119
0.028 ->   0.102925
0.029 ->   0.104705
0.030 ->   0.106461
0.031 ->   0.108194
0.032 ->   0.109903
0.033 ->   0.111591
0.034 ->   0.113257
0.035 ->   0.114903
0.036 ->   0.116528
0.037 ->   0.118134
0.038 ->   0.119722
0.039 ->   0.121290
0.040 ->   0.197197
0.041 ->   0.124376
0.042 ->   0.125893
0.043 ->   0.127395
0.044 ->   0.128879
0.045 ->   0.130350
0.046 ->   0.131805
0.047 ->   0.133244
0.048 ->   0.134671
0.049 ->   0.136082
